In [1]:
# Загрузка библиотек
import pandas as pd
import numpy as np

from funcs import fill_with_mode, clamp

In [2]:
# Загрузка датасета
db2 = pd.read_excel('Database_2.xlsx')
db2.head()

,Nanoparticle,Type: Organic (O)/inorganic (I),coat,Diameter (nm),Concentration μM,Zeta potential (mV),Cells,Cell line (L)/primary cells (P),Human(H)/Animal(A) cells,Animal?,...,Test,Test indicator,Biochemical metric,% Cell viability,Interference checked (Y/N),Colloidal stability checked (Y/N),Positive control (Y/N),Publication year,Particle ID,Reference DOI
0,CeO2,I,NaN,20.3,0.002478,NaN,L929,L,A,Mouse,...,MTT,tetrazolium salt,cell metabolic activity,105.58918,N,N,N,2015,1,10.1016/j.ceramint.2014.09.095
1,CeO2,I,NaN,20.3,0.004956,NaN,L929,L,A,Mouse,...,MTT,tetrazolium salt,cell metabolic activity,103.02621,N,N,N,2015,1,10.1016/j.ceramint.2014.09.095
2,CeO2,I,NaN,20.3,0.009911,NaN,L929,L,A,Mouse,...,MTT,tetrazolium salt,cell metabolic activity,104.85748,N,N,N,2015,1,10.1016/j.ceramint.2014.09.095
3,CeO2,I,NaN,20.3,0.019823,NaN,L929,L,A,Mouse,...,MTT,tetrazolium salt,cell metabolic activity,100.09670,N,N,N,2015,1,10.1016/j.ceramint.2014.09.095
4,CeO2,I,NaN,20.3,0.039645,NaN,L929,L,A,Mouse,...,MTT,tetrazolium salt,cell metabolic activity,97.53237,N,N,N,2015,1,10.1016/j.ceramint.2014.09.095


In [3]:
# Ненужные колонки
db2.drop(['Publication year', 'Particle ID', 'Reference DOI'], axis=1, inplace=True)

In [4]:
# Информация
db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Nanoparticle                        2896 non-null   object 
 1   Type: Organic (O)/inorganic (I)     2896 non-null   object 
 2   coat                                1052 non-null   object 
 3   Diameter (nm)                       2896 non-null   float64
 4   Concentration μM                    2896 non-null   float64
 5   Zeta potential (mV)                 1261 non-null   float64
 6   Cells                               2896 non-null   object 
 7   Cell line (L)/primary cells (P)     2896 non-null   object 
 8   Human(H)/Animal(A) cells            2896 non-null   object 
 9   Animal?                             651 non-null    object 
 10  Cell morphology                     2895 non-null   object 
 11  Cell age: embryonic (E), Adult (A)  2895 no

In [5]:
# Очистка столбца с животными
db2['Animal?'].fillna('unknown', inplace=True)
db2['Animal?'].unique()

array(['Mouse', 'unknown', 'Rat', 'Pig',
       'Monkey (Cercopithecus aethiops)', 'Canine', 'hamster', 'rat',
       'Cow, Bos taurus', 'Mice'], dtype=object)

In [6]:
db2['Animal?'] = db2['Animal?'].replace({
    'Mouse': 'rat',
    'Mice': 'rat',
    'Cow, Bos taurus': 'cow',
    'Canine': 'dog'
})

In [7]:
db2['Animal'] = db2['Animal?'].apply(lambda x: x.lower().split()[0])
del db2['Animal?']

In [8]:
# Очистка столбца с целевой переменной
fill_with_mode(db2['% Cell viability'])

In [9]:
db2['Type: Organic (O)/inorganic (I)'].unique()

array(['I', 'O', 0], dtype=object)

In [10]:
# Очистка столбца типа наночастиц
db2['is_inorganic'] = db2['Type: Organic (O)/inorganic (I)'].apply(
    lambda x: 1 if x == 'I' else 0
)
del db2['Type: Organic (O)/inorganic (I)']

In [11]:
# Очистка столбца с зета-потенциалом наночастиц
db2['Zeta potential (mV)'].unique()

array([   nan,   0.  , -39.67, -26.  ,  43.8 , -44.6 ,  44.1 ,  45.2 ,
        17.7 ,  12.1 , -23.5 , -27.7 , -17.6 ,  -9.96, -27.8 ,  -8.  ,
       -40.5 , -29.  , -30.  , -42.  , -27.  , -11.  ,  87.  ,  58.  ,
        79.  ,  30.  ,  32.  , -48.  , -35.9 , -26.8 ,  21.7 ,  -7.6 ,
       -20.  , -33.33, -23.1 , -18.4 ,  56.  ,  -5.38,   8.3 ,  21.5 ,
        26.5 ,  -8.6 ,  10.  ,  -5.  , -24.  , -19.9 ,  -9.8 , -22.6 ,
       -12.3 , -14.6 ,  34.6 ,  30.5 ,  28.8 ,  25.4 ,  15.4 ,  14.3 ,
        43.9 ,  47.  ])

In [12]:
db2['Zeta potential (mV)'].fillna(db2['Zeta potential (mV)'].median(), inplace=True)

In [13]:
# Переименование столбца с возрастом клетки
db2['Cell age'] = db2['Cell age: embryonic (E), Adult (A)']
del db2['Cell age: embryonic (E), Adult (A)']

In [14]:
# Очистка других столбцов
for c in ['Cell age', 'Cell morphology', 'Test', 'Test indicator', 'Biochemical metric']:
    fill_with_mode(db2[c])

In [15]:
# Очистка столбца 'coat'
db2['coat'].fillna('nothing', inplace=True)

In [16]:
# Очистка столбца с соединениями наночастиц
db2['Nanoparticle'].unique()

array(['CeO2', 'Se', 'CuO', 'Copper Oxide', 'CuS', 'ZnO', 'Zinc oxide',
       'Iron oxide', 'TiO2', 'Ti', 'Cr', 'Co', 'Co3O4', 'CdO', 'MnO',
       'Bi', 'Mo', 'Pt', 'Hydroxyapatite', 'Cu2O', 'MgO', 'Al2O3', 'QDs',
       'SiO2', 'Au', 'Ag', 'Eudragit RL', 'Dendrimer', 'PLGA',
       'Polystyrene', 'Liposomes', 'SLN', 'Carbon Nanotubes', 'Carbon NP',
       'Chitosan'], dtype=object)

In [17]:
db2['Nanoparticle'].replace({
    'Zinc oxide': 'ZnO',
    'Copper Oxide': 'CuO',
    'Iron oxide': 'Fe2O3'
}, inplace=True)

In [18]:
# Сохранение таблицы
db2.to_excel('proc_db2.xlsx', index=False)